<span type="title">Lambda Functions and Stream API</span> | <span type="update">2018-10-22</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Java 8 的新特性：类似于函数式编程的 Lambda API 和 基于 Lambda 的 Stream 流处理 API。此外，也会涉及 Optional 接口和并发编程等方面。</p></span>

# Lambda Functions

函数式编程允许将函数作为入参传递，注意，这里的入参并不是指的函数返回的值作为入参，而是说，函数本身作为入参。Lambda 表达式类似于函数式编程，它的形式为 `(x, y) -> x * y`，你可以把它作为一个参数传递到一个方法中。你可能注意到，这个函数仅仅定义了一组计算方法，但是没有提供数据，这是因为我们将数据和策略进行了区分，函数提供的是计算数值的策略，类似于策略模式。

在稍后的例子中，将会看到更多使用函数式编程简化 OOP 设计模式的例子。在一些情况下，它们被称之为语法糖，但不论是从形式还是阅读上，函数式编程都十分优雅，所编写的代码更加简洁。

函数式编程是一种对于 OOP 设计模式的简化，可以由下面这个例子看出：

为了得到一个接口的对象，我们必须手动创建这个接口的一个实例：

```java
Comparator<Integer> comparator = new Comparator<Integer>() {
    public int compare(Integer o1, Integer o2) {return Integer.compare(o1,o2);}
};
TreeSet<Integer> treeSet = new TreeSet<>(comparator);

Comparator<Integer> comparator = (x, y) -> Integer.compare(x,y);
TreeSet<Integer> treeSet = new TreeSet<>(comparator);
```

可以看到，因为 Java 的纯 OO 风格，我们必须为一个方法提供一个类，尽管我们在上面已经使用了匿名内部类简化了创造一个类的步骤（传统过程是创建一个 .class 文件，这个文件包含一个新的类），但是，除了核心的 `Integer.compare(o1,o2)`，其余所有的代码都是冗余的。

而函数式的风格如下所示：我们直接可以创建一个 “函数”，作为一个变量？真的可以吗？当然不可以，这只是看起来像 FP 的 OOP 语法糖，在编译期还是会被解包成为匿名内部类，编译 .class 文件，然后交给 JVM 执行。

尽管如此，这种写法看起来更清晰，优雅，可能唯一的缺点就是，如果你不熟悉 Comparator 接口，你可能不知道这里的 x, y 是什么类型，因为这一切都是 编译器自动推断得来的。

## 面向对象、设计模式和匿名内部类

Lambda 是一种用来促进代码复用的技术，和 OOP 的设计模式一样，或者说，Lambda 表达式能够以 FP 的方式来实现设计模式，而不用传统 OOP 的方式，这种方式间接优雅。

首先，我们要看一看设计模式的使用场景：

首先定义一个 Employee 类，这个类是一个 POJO，包括以下属性：

```java
public class Employee {
    private String name;
    private Integer age;
    private Integer salary; 
    ... 
}
```

然后伪造一个员工类的集合：

```java
List<Employee> employeeList = Arrays.asList(
    new Employee("A",118,2323),new Employee("B",618,23223),new Employee("C",158,2321),
    new Employee("D",118,4543),new Employee("E",128,2466));
```

如果我们现在希望从这一组员工中找年龄大于120岁的，我们要编写一个方法：

```java
public List<Employee> filterEmployees(List<Employee> list) {
    List<Employee> list1 = new ArrayList<>();
    for (Employee e: list) {if (e.getAge() >= 120) {list1.add(e);}}
    return list1;
}
```

然后调用这个方法： `filterEmployees(employeeList)`

这是纯粹的 OOP 设计，但是问题是，我们没有区分可变和不可变的，如果我们需要更改需求，找到年龄大于130岁的员工，那么就不得不重写上述方法，如果我们要过滤年龄的同时需要排序、过滤薪水，那么问题就更加复杂了，为此，我们有了设计模式。设计模式的目的就是为了促进 OOP 的代码复用。

首先我们要面向接口编程，因此要有一个接口 `public interface Filter<T> {boolean need(T t);}`, 这个接口可以为我们提供“策略模式”的策略类所必须的方法 `need`。之后，编写这个接口的不同实现：

```java
class AgeFilter implements Filter<Employee> {
    public boolean need(Employee employee) {return employee.getAge() >= 120;}
}
```

然后重写原来的方法：

```java
public List<Employee> filterEmpInPattern(List<Employee> list, Filter<Employee> filter) {
    List<Employee> list1 = new ArrayList<>();
    for (Employee e: list) {if (filter.need(e)) list1.add(e);}
    return list1;
}
```

这样的话，当我们有了新的需求，就写一个新的策略，使用现在重构过的方法，就可以在相同的数据源上应用不同的策略了。这是“区分变化的和不可变化的”的一个绝佳实例，也就是设计模式中使用最多的“策略模式”。初看起来，这种方式需要的代码行数更多，文件更多，操作更复杂，但是，所有的设计模式都不是为了写 1 + 1 = 2 这种级别的程序服务的，当代码急剧增长，设计模式是唯一能够拯救 OOP 的重要思想和最佳的实践。

以上的设计模式，按照匿名内部类的方式，可以简写成为：

```java
filterEmpInPattern(employeeList, new Filter<Employee>() {
    public boolean need(Employee employee) {return employee.getAge() >= 120;}})
```

这样看上去已经非常的清爽了，但是，里面真正有用的逻辑其实还只是一句话 `employee.getAge() >= 120;`

我们看看 Java 8 Lambda 表达式的写法： `filterEmpInPattern(employeeList, employee -> employee.getAge() >= 120)`。极其的间接和直白。

其实，使用基于 Lambda 的 Stream 流，还可以这样写：

```java
employeeList.stream().filter(e -> e.getAge() >= 120).limit(2).forEach(System.out::println);
```

看起来更棒了。

## Lambda 表达式基础

 `->` 操作符称为箭头操作符/Lambda操作符，比如：`(e) -> { e.getAge() >= 120 }`
 
表达式的左侧是参数列表，参数列表即实现所需的方法参数列表。

表达式的右侧是表达式所需执行的功能，即 Lambda 体。Lambda体即对于接口的实现。
 
因为接口可能无参无返回值、有单个参，无返回值、有多个参，无返回值。以及三种情况有返回值六种情况。

**没有参数，无返回值**

无参无返回值类似于 ` () -> System.out.print("Hello");` 无参不需要提供参数，但是要写小括号。Lambda 体提供了此方法的动作，在这里是打印一个单词。匿名内部类和Lambda内部引用的本地变量本质上还是 final 方法，不过不用加，但是不可 ++。如下所示：

```java
int a = 8;
Runnable runnable = () -> System.out.println("Hello, Lambda in J" + a);
runnable.run();
```

在上面的例子中，runnable 变量是一个 Runnable 类型的对象，只不过我们没有采用 OOP 的方式创建对象，而是采用了 FP 的方式。这类似于 FP “交给子类”设计模式的写法。因为我们得到的是一个 Runnable 类型的对象，自然可以调用相应的方法。

注意，这里的 a 变量虽然不用写 final（J8），但是在 Lambda 体重默认含义依旧是不可能更改的 final 类型。如果在 Lambda 体中写 a++ 就会报错。

**有一个参数，无返回值**

类似 `(e) -> {System.out.print(e)};`，有一个参数，则在小括号写一个参数的引用，注意：

- 参数列表可以提供类型信息，也可以不提供
- 当参数列表只有一个参数，可以省略小括号。
- 当实现只有一条语句，可以省略花括号。

```java
Consumer<Employee> consumer = employee -> System.out.println(employee);
consumer.accept(new Employee("Marvin",null,null));
```

**有多个参数，无返回值**

对于有多个参数，无返回值： 

- 如果有多条语句，使用 {}，如果只有一条语句，可以省略 {}。

`(a, b) -> {System.out.print(a + b)};`

`(a, b) -> System.out.print(a + b)`


**有零个/单个/多个参数，有返回值**

参数列表需要注意：

- 对于有一个参数，有返回值，参数小括号可省略

Lambda 体需要注意：

- 如果结构体有多条语句，则使用 {}。
- 如果结构体只有一条语句，则 return 可以不写。

下列语句等同：

`(a, b) -> { if (a > b) return a; else return b; }` 

`(c, d) -> ((c > d) ? c : d);`

## z

左边为参数列表，类型自动推断，可以不写类型。如果没有参数，使用 （），如果有一个参数，则可以省略（）。如果有多个参数，则不可省略（）。

右边是 Lambda 体，对于单条语句，可以不写 {}，对于多条语句，不可不写 {}。对于有返回值的单条语句，可以省略 return 和 {}，对于有返回值的多条语句，均不可省略。

概括为：左右遇一括号省，左侧推断类型省。能省则省。

Lambda 表达式需要函数式接口（接口中只有一个抽象方法）的支持。可以使用 @FunctionalInterface 修饰接口，编译器会检查此接口是否为函数式接口。

常见用法：声明一个函数式接口，然后提供一个策略方法。一般不是这样直接声明一个“匿名抽象类”，然后调用其方法
//这种用法可以看作是交给子类的设计模式的实现，一般我们使用组合而不是继承
MyFunction function = d -> d * 1000;
System.out.println(function.operation(100));
//正宗的用法是为函数式接口提供一个能够接受策略的方法/构造器，然后调用此构造器返回值。
//这种用法可以看作是策略模式的FP实现，最为常用。 此外，这个看起来更像 FP。
System.out.println(doMath(100, d -> d * 1000));